In [ ]:
!pip install pycaret

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install lime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283839 sha256=3940023f62d89f1d958498c377f7fa3a2aaf66a5b936d4422ee61ebfe429b5fd
  Stored in directory: /root/.cache/pip/wheels/fd/a2/af/9ac0a1a85a27f314a06b39e1f492bee1547d52549a4606ed89
Successfully built lime


In [ ]:
!pip install shap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 572.6/572.6 kB 10.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from statsmodels.stats.proportion import proportions_ztest
from scipy.stats import ttest_1samp, shapiro
from sklearn.feature_selection import chi2

from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from sklearn.metrics import recall_score, make_scorer, roc_auc_score, confusion_matrix, classification_report, ConfusionMatrixDisplay

from sklearn.model_selection import GridSearchCV, RepeatedStratifiedKFold

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

from graphviz import Source
from sklearn.tree import export_graphviz
from sklearn import tree
from IPython.display import SVG, display
import shap

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Exp 1: optimal windsor old nulls

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/ml1_final_project/data_cardiovascular_risk.csv')

In [ ]:
columns_to_convert = ['sex', 'education', 'is_smoking', 'BPMeds', 'prevalentStroke', 'prevalentHyp', 'diabetes', 'TenYearCHD']
df[columns_to_convert] = df[columns_to_convert].astype('object')

In [ ]:
categorical_columns = df.select_dtypes(include='object').columns
numerical_columns = df.select_dtypes(include=['float', 'int']).columns

In [ ]:
categorical_columns

Index(['education', 'sex', 'is_smoking', 'BPMeds', 'prevalentStroke',
       'prevalentHyp', 'diabetes', 'TenYearCHD'],
      dtype='object')

In [ ]:
numerical_columns

Index(['id', 'age', 'cigsPerDay', 'totChol', 'sysBP', 'diaBP', 'BMI',
       'heartRate', 'glucose'],
      dtype='object')

In [ ]:
null_counts = df.isnull().sum().sort_values(ascending=False)

# Calculate the percentage of null values in each numerical column
null_percentages = (null_counts / len(df)) * 100

# Create a DataFrame with null counts and percentages
null_table = pd.DataFrame({
    'Null Count': null_counts,
    'Null Percentage': null_percentages
})

# Display the null table
print(null_table)

                 Null Count  Null Percentage
glucose                 304         8.967552
education                87         2.566372
BPMeds                   44         1.297935
totChol                  38         1.120944
cigsPerDay               22         0.648968
BMI                      14         0.412979
heartRate                 1         0.029499
id                        0         0.000000
diaBP                     0         0.000000
sysBP                     0         0.000000
prevalentHyp              0         0.000000
diabetes                  0         0.000000
age                       0         0.000000
prevalentStroke           0         0.000000
is_smoking                0         0.000000
sex                       0         0.000000
TenYearCHD                0         0.000000


In [ ]:
# Remove rows where 'bpmeds' column is null
df = df.dropna(subset=['BPMeds'])

imputing_median1 = df[df['is_smoking'] == 'YES']['cigsPerDay'].median()
df['cigsPerDay'] = df['cigsPerDay'].fillna(imputing_median1)

for var in ['totChol', 'BMI', 'heartRate']:
  imputing_median = df[var].median()
  df[var]  = df[var].fillna(imputing_median)

df.drop('id', axis=1, inplace=True)
df.drop('education', axis=1, inplace=True)
df.drop(['is_smoking', 'prevalentStroke'], axis=1, inplace=True)

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer


# fit and transform the imputer on your data
imputer = IterativeImputer(random_state = 0)
imputer.fit(df[['glucose']])

df[['glucose']] = imputer.transform(df[['glucose']])


In [ ]:
df.isnull().sum()

age             0
sex             0
cigsPerDay      0
BPMeds          0
prevalentHyp    0
diabetes        0
totChol         0
sysBP           0
diaBP           0
BMI             0
heartRate       0
glucose         0
TenYearCHD      0
dtype: int64

In [ ]:
df = pd.get_dummies(df, drop_first=True)

In [ ]:
X = df.drop(['TenYearCHD_1'], axis = 1)
y = df['TenYearCHD_1']

In [ ]:
X_winsorized = X.copy()

In [ ]:
from scipy.stats.mstats import winsorize
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import roc_auc_score

# Define the range of winsorizing limits to try
lower_limits_range = [0, 0.01, 0.05, 0.1, 0.15, 0.1, 0.15, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95]
upper_limits_range = [0, 0.01, 0.05, 0.1, 0.15, 0.1, 0.15, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95]

# Initialize variables to store the optimal limits and corresponding AUC score
optimal_limits = {}
optimal_auc = 0

# Iterate over the columns
for column in X.columns:
    # Initialize variables to store the optimal lower and upper limits for the current column
    optimal_lower_limit = 0
    optimal_upper_limit = 0
    
    # Iterate over the lower limits range
    for lower_limit in lower_limits_range:
        # Iterate over the upper limits range
        for upper_limit in upper_limits_range:
            # Apply winsorizing on the column with the current lower and upper limits
            X_winsorized = X.copy()
            X_winsorized[column] = winsorize(X_winsorized[column], limits=(lower_limit, upper_limit))
        
            # Split the winsorized data into training and testing sets
            X_train, X_test, y_train, y_test = train_test_split(X_winsorized, y, test_size=0.2, random_state=42)
        
            # Train Gaussian Naive Bayes
            gnb = GaussianNB()
            gnb.fit(X_train, y_train)
        
            # Make predictions and calculate AUC
            y_pred = gnb.predict_proba(X_test)[:, 1]
            auc = roc_auc_score(y_test, y_pred)
        
            # Check if the current AUC is higher than the optimal AUC
            if auc > optimal_auc:
                optimal_auc = auc
                optimal_lower_limit = lower_limit
                optimal_upper_limit = upper_limit
    
    # Store the optimal limits for the current column
    optimal_limits[column] = (optimal_lower_limit, optimal_upper_limit)

# Print the optimal limits for each column
for column, (lower_limit, upper_limit) in optimal_limits.items():
    print(f"Optimal limits for column '{column}': Lower Limit = {lower_limit}, Upper Limit = {upper_limit}")

# Print the overall optimal AUC score
print("Optimal AUC:", optimal_auc)


Optimal limits for column 'age': Lower Limit = 0, Upper Limit = 0
Optimal limits for column 'cigsPerDay': Lower Limit = 0.7, Upper Limit = 0.01
Optimal limits for column 'totChol': Lower Limit = 0, Upper Limit = 0
Optimal limits for column 'sysBP': Lower Limit = 0, Upper Limit = 0
Optimal limits for column 'diaBP': Lower Limit = 0.8, Upper Limit = 0
Optimal limits for column 'BMI': Lower Limit = 0, Upper Limit = 0
Optimal limits for column 'heartRate': Lower Limit = 0, Upper Limit = 0
Optimal limits for column 'glucose': Lower Limit = 0.7, Upper Limit = 0.2
Optimal limits for column 'sex_M': Lower Limit = 0, Upper Limit = 0
Optimal limits for column 'BPMeds_1.0': Lower Limit = 0, Upper Limit = 0
Optimal limits for column 'prevalentHyp_1': Lower Limit = 0, Upper Limit = 0.4
Optimal limits for column 'diabetes_1': Lower Limit = 0, Upper Limit = 0
Optimal AUC: 0.7269016018306635


Optimal limits for column 'glucose': Lower Limit = 0.15, Upper Limit = 0.15
Optimal limits for column 'totChol': Lower Limit = 0, Upper Limit = 0
Optimal limits for column 'BMI': Lower Limit = 0, Upper Limit = 0
Optimal limits for column 'heartRate': Lower Limit = 0, Upper Limit = 0
Optimal AUC: 0.7197803203661327


# Exp 2: optimal windsor iterative imputer on 3 also nulls

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/ml1_final_project/data_cardiovascular_risk.csv')

columns_to_convert = ['sex', 'education', 'is_smoking', 'BPMeds', 'prevalentStroke', 'prevalentHyp', 'diabetes', 'TenYearCHD']
df[columns_to_convert] = df[columns_to_convert].astype('object')

categorical_columns = df.select_dtypes(include='object').columns
numerical_columns = df.select_dtypes(include=['float', 'int']).columns



In [ ]:

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Remove rows where 'bpmeds' column is null
df = df.dropna(subset=['BPMeds'])

imputing_median1 = df[df['is_smoking'] == 'YES']['cigsPerDay'].median()
df['cigsPerDay'] = df['cigsPerDay'].fillna(imputing_median1)

df.drop('id', axis=1, inplace=True)
df.drop('education', axis=1, inplace=True)
df.drop(['is_smoking', 'prevalentStroke'], axis=1, inplace=True)

for var in ['totChol', 'BMI', 'heartRate']:
  imputer = IterativeImputer(random_state=0)
  df[[var]] = imputer.fit_transform(df[[var]])




# fit and transform the imputer on your data
imputer = IterativeImputer(random_state = 0)
imputer.fit(df[['glucose']])

df[['glucose']] = imputer.transform(df[['glucose']])


In [ ]:
df.isnull().sum()

age             0
sex             0
cigsPerDay      0
BPMeds          0
prevalentHyp    0
diabetes        0
totChol         0
sysBP           0
diaBP           0
BMI             0
heartRate       0
glucose         0
TenYearCHD      0
dtype: int64

In [ ]:
df = pd.get_dummies(df, drop_first=True)

In [ ]:
X = df.drop(['TenYearCHD_1'], axis = 1)
y = df['TenYearCHD_1']

In [ ]:
X_winsorized = X.copy()

In [ ]:
from scipy.stats.mstats import winsorize
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import roc_auc_score

# Define the range of winsorizing limits to try
lower_limits_range = [0, 0.01, 0.05, 0.1, 0.15, 0.1, 0.15, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95]
upper_limits_range = [0, 0.01, 0.05, 0.1, 0.15, 0.1, 0.15, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95]

# Initialize variables to store the optimal limits and corresponding AUC score
optimal_limits = {}
optimal_auc = 0

# Iterate over the columns
for column in X.columns:
    # Initialize variables to store the optimal lower and upper limits for the current column
    optimal_lower_limit = 0
    optimal_upper_limit = 0
    
    # Iterate over the lower limits range
    for lower_limit in lower_limits_range:
        # Iterate over the upper limits range
        for upper_limit in upper_limits_range:
            # Apply winsorizing on the column with the current lower and upper limits
            X_winsorized = X.copy()
            X_winsorized[column] = winsorize(X_winsorized[column], limits=(lower_limit, upper_limit))
        
            # Split the winsorized data into training and testing sets
            X_train, X_test, y_train, y_test = train_test_split(X_winsorized, y, test_size=0.2, random_state=42)
        
            # Train Gaussian Naive Bayes
            gnb = GaussianNB()
            gnb.fit(X_train, y_train)
        
            # Make predictions and calculate AUC
            y_pred = gnb.predict_proba(X_test)[:, 1]
            auc = roc_auc_score(y_test, y_pred)
        
            # Check if the current AUC is higher than the optimal AUC
            if auc > optimal_auc:
                optimal_auc = auc
                optimal_lower_limit = lower_limit
                optimal_upper_limit = upper_limit
    
    # Store the optimal limits for the current column
    optimal_limits[column] = (optimal_lower_limit, optimal_upper_limit)

# Print the optimal limits for each column
for column, (lower_limit, upper_limit) in optimal_limits.items():
    print(f"Optimal limits for column '{column}': Lower Limit = {lower_limit}, Upper Limit = {upper_limit}")

# Print the overall optimal AUC score
print("Optimal AUC:", optimal_auc)


Optimal limits for column 'age': Lower Limit = 0, Upper Limit = 0
Optimal limits for column 'cigsPerDay': Lower Limit = 0.7, Upper Limit = 0.01
Optimal limits for column 'totChol': Lower Limit = 0, Upper Limit = 0
Optimal limits for column 'sysBP': Lower Limit = 0, Upper Limit = 0
Optimal limits for column 'diaBP': Lower Limit = 0.8, Upper Limit = 0
Optimal limits for column 'BMI': Lower Limit = 0, Upper Limit = 0
Optimal limits for column 'heartRate': Lower Limit = 0, Upper Limit = 0
Optimal limits for column 'glucose': Lower Limit = 0.7, Upper Limit = 0.2
Optimal limits for column 'sex_M': Lower Limit = 0, Upper Limit = 0
Optimal limits for column 'BPMeds_1.0': Lower Limit = 0, Upper Limit = 0
Optimal limits for column 'prevalentHyp_1': Lower Limit = 0, Upper Limit = 0.4
Optimal limits for column 'diabetes_1': Lower Limit = 0, Upper Limit = 0
Optimal AUC: 0.7270663615560642


Optimal limits for column 'glucose': Lower Limit = 0.15, Upper Limit = 0.15
Optimal limits for column 'totChol': Lower Limit = 0, Upper Limit = 0
Optimal limits for column 'BMI': Lower Limit = 0, Upper Limit = 0
Optimal limits for column 'heartRate': Lower Limit = 0, Upper Limit = 0
Optimal AUC: 0.7197803203661327


# Exp 3: optimal windsor KNN imputer

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/ml1_final_project/data_cardiovascular_risk.csv')

columns_to_convert = ['sex', 'education', 'is_smoking', 'BPMeds', 'prevalentStroke', 'prevalentHyp', 'diabetes', 'TenYearCHD']
df[columns_to_convert] = df[columns_to_convert].astype('object')

categorical_columns = df.select_dtypes(include='object').columns
numerical_columns = df.select_dtypes(include=['float', 'int']).columns



In [ ]:

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Remove rows where 'bpmeds' column is null
df = df.dropna(subset=['BPMeds'])

imputing_median1 = df[df['is_smoking'] == 'YES']['cigsPerDay'].median()
df['cigsPerDay'] = df['cigsPerDay'].fillna(imputing_median1)

df.drop('id', axis=1, inplace=True)
df.drop('education', axis=1, inplace=True)
df.drop(['is_smoking', 'prevalentStroke'], axis=1, inplace=True)

from sklearn.impute import KNNImputer

for var in ['totChol', 'BMI', 'heartRate']:
    imputer = KNNImputer(n_neighbors=5)
    df[[var]] = imputer.fit_transform(df[[var]])

imputer = KNNImputer(n_neighbors=5)
df[['glucose']] = imputer.fit_transform(df[['glucose']])


df[['glucose']] = imputer.transform(df[['glucose']])


In [ ]:
df.isnull().sum()

age             0
sex             0
cigsPerDay      0
BPMeds          0
prevalentHyp    0
diabetes        0
totChol         0
sysBP           0
diaBP           0
BMI             0
heartRate       0
glucose         0
TenYearCHD      0
dtype: int64

In [ ]:
df = pd.get_dummies(df, drop_first=True)

In [ ]:
X = df.drop(['TenYearCHD_1'], axis = 1)
y = df['TenYearCHD_1']

In [ ]:
X_winsorized = X.copy()

In [ ]:
from scipy.stats.mstats import winsorize
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import roc_auc_score

# Define the range of winsorizing limits to try
lower_limits_range = [0, 0.01, 0.05, 0.1, 0.15, 0.1, 0.15, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95]
upper_limits_range = [0, 0.01, 0.05, 0.1, 0.15, 0.1, 0.15, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95]

# Initialize variables to store the optimal limits and corresponding AUC score
optimal_limits = {}
optimal_auc = 0

# Iterate over the columns
for column in X.columns:
    # Initialize variables to store the optimal lower and upper limits for the current column
    optimal_lower_limit = 0
    optimal_upper_limit = 0
    
    # Iterate over the lower limits range
    for lower_limit in lower_limits_range:
        # Iterate over the upper limits range
        for upper_limit in upper_limits_range:
            # Apply winsorizing on the column with the current lower and upper limits
            X_winsorized = X.copy()
            X_winsorized[column] = winsorize(X_winsorized[column], limits=(lower_limit, upper_limit))
        
            # Split the winsorized data into training and testing sets
            X_train, X_test, y_train, y_test = train_test_split(X_winsorized, y, test_size=0.2, random_state=42)
        
            # Train Gaussian Naive Bayes
            gnb = GaussianNB()
            gnb.fit(X_train, y_train)
        
            # Make predictions and calculate AUC
            y_pred = gnb.predict_proba(X_test)[:, 1]
            auc = roc_auc_score(y_test, y_pred)
        
            # Check if the current AUC is higher than the optimal AUC
            if auc > optimal_auc:
                optimal_auc = auc
                optimal_lower_limit = lower_limit
                optimal_upper_limit = upper_limit
    
    # Store the optimal limits for the current column
    optimal_limits[column] = (optimal_lower_limit, optimal_upper_limit)

# Print the optimal limits for each column
for column, (lower_limit, upper_limit) in optimal_limits.items():
    print(f"Optimal limits for column '{column}': Lower Limit = {lower_limit}, Upper Limit = {upper_limit}")

# Print the overall optimal AUC score
print("Optimal AUC:", optimal_auc)


Optimal limits for column 'age': Lower Limit = 0, Upper Limit = 0
Optimal limits for column 'cigsPerDay': Lower Limit = 0.7, Upper Limit = 0.01
Optimal limits for column 'totChol': Lower Limit = 0, Upper Limit = 0
Optimal limits for column 'sysBP': Lower Limit = 0, Upper Limit = 0
Optimal limits for column 'diaBP': Lower Limit = 0.8, Upper Limit = 0
Optimal limits for column 'BMI': Lower Limit = 0, Upper Limit = 0
Optimal limits for column 'heartRate': Lower Limit = 0, Upper Limit = 0
Optimal limits for column 'glucose': Lower Limit = 0.7, Upper Limit = 0.2
Optimal limits for column 'sex_M': Lower Limit = 0, Upper Limit = 0
Optimal limits for column 'BPMeds_1.0': Lower Limit = 0, Upper Limit = 0
Optimal limits for column 'prevalentHyp_1': Lower Limit = 0, Upper Limit = 0.4
Optimal limits for column 'diabetes_1': Lower Limit = 0, Upper Limit = 0
Optimal AUC: 0.7270663615560642


# Exp 4: Imputation using KNN

In [ ]:
df.isnull().sum()

age             0
sex             0
cigsPerDay      0
BPMeds          0
prevalentHyp    0
diabetes        0
totChol         0
sysBP           0
diaBP           0
BMI             0
heartRate       0
glucose         0
TenYearCHD      0
dtype: int64

In [ ]:
df = pd.get_dummies(df, drop_first=True)

In [ ]:
X = df.drop(['TenYearCHD_1'], axis = 1)
y = df['TenYearCHD_1']

In [ ]:
X_winsorized = X.copy()

In [ ]:
from scipy.stats.mstats import winsorize
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import roc_auc_score

# Define the range of winsorizing limits to try
lower_limits_range = [0, 0.01, 0.05, 0.1, 0.15, 0.1, 0.15, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95]
upper_limits_range = [0, 0.01, 0.05, 0.1, 0.15, 0.1, 0.15, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95]

# Initialize variables to store the optimal limits and corresponding AUC score
optimal_limits = {}
optimal_auc = 0

# Iterate over the columns
for column in X.columns:
    # Initialize variables to store the optimal lower and upper limits for the current column
    optimal_lower_limit = 0
    optimal_upper_limit = 0
    
    # Iterate over the lower limits range
    for lower_limit in lower_limits_range:
        # Iterate over the upper limits range
        for upper_limit in upper_limits_range:
            # Apply winsorizing on the column with the current lower and upper limits
            X_winsorized = X.copy()
            X_winsorized[column] = winsorize(X_winsorized[column], limits=(lower_limit, upper_limit))
        
            # Split the winsorized data into training and testing sets
            X_train, X_test, y_train, y_test = train_test_split(X_winsorized, y, test_size=0.2, random_state=42)
        
            # Train Gaussian Naive Bayes
            gnb = GaussianNB()
            gnb.fit(X_train, y_train)
        
            # Make predictions and calculate AUC
            y_pred = gnb.predict_proba(X_test)[:, 1]
            auc = roc_auc_score(y_test, y_pred)
        
            # Check if the current AUC is higher than the optimal AUC
            if auc > optimal_auc:
                optimal_auc = auc
                optimal_lower_limit = lower_limit
                optimal_upper_limit = upper_limit
    
    # Store the optimal limits for the current column
    optimal_limits[column] = (optimal_lower_limit, optimal_upper_limit)

# Print the optimal limits for each column
for column, (lower_limit, upper_limit) in optimal_limits.items():
    print(f"Optimal limits for column '{column}': Lower Limit = {lower_limit}, Upper Limit = {upper_limit}")

# Print the overall optimal AUC score
print("Optimal AUC:", optimal_auc)


Optimal limits for column 'age': Lower Limit = 0, Upper Limit = 0
Optimal limits for column 'cigsPerDay': Lower Limit = 0.7, Upper Limit = 0.01
Optimal limits for column 'totChol': Lower Limit = 0, Upper Limit = 0
Optimal limits for column 'sysBP': Lower Limit = 0, Upper Limit = 0
Optimal limits for column 'diaBP': Lower Limit = 0.8, Upper Limit = 0
Optimal limits for column 'BMI': Lower Limit = 0, Upper Limit = 0
Optimal limits for column 'heartRate': Lower Limit = 0, Upper Limit = 0
Optimal limits for column 'glucose': Lower Limit = 0.7, Upper Limit = 0.2
Optimal limits for column 'sex_M': Lower Limit = 0, Upper Limit = 0
Optimal limits for column 'BPMeds_1.0': Lower Limit = 0, Upper Limit = 0
Optimal limits for column 'prevalentHyp_1': Lower Limit = 0, Upper Limit = 0.4
Optimal limits for column 'diabetes_1': Lower Limit = 0, Upper Limit = 0
Optimal AUC: 0.7270663615560642
